In [11]:
import pandas as pd 

stroke_dft  = pd.read_csv('healthcare_dataset.csv')
stroke_dft.head(5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [12]:
stroke_dft.dtypes


id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [13]:
#  fill in  missing value using the median value of the column
stroke_dft['bmi'] = stroke_dft['bmi'].fillna(stroke_dft['bmi'].median())
stroke_dft['bmi_category'] = stroke_dft['bmi'].apply(lambda bmi: 'Underweight' if bmi < 20   else 'Normal' if bmi < 25 
    else 'Overweight' if bmi < 30     else 'Obese' )
stroke_dft['age_catergory'] = stroke_dft['age'].apply( lambda age: 'Child' if age < 18   else 'Young_Adult' if age < 35 
    else 'Adult' if age < 60    else 'Senior')
stroke_dft['glucose_category'] = stroke_dft['avg_glucose_level'].apply( lambda gl: 'Low' if gl < 70  else 'Normal' if gl <= 140 
    else 'Prediabetic' if gl<= 200    else 'Diabetic' )
stroke_dft['ever_married_bool'] = stroke_dft['ever_married'].map({'Yes': 1, 'No': 0})
stroke_dft['residence_type_bool'] = stroke_dft['Residence_type'].map({'Urban': 1, 'Rural': 0})
stroke_dft.drop(columns=['id'], inplace=True)


stroke_dft_cleaned = pd.get_dummies(stroke_dft, columns=['gender','age_catergory','bmi_category','work_type','glucose_category','smoking_status'], prefix=['g','age','bmi','work_type','gl_catergory','smoking_status'])

# Save cleaned dataset for testing purposes 
stroke_dft_cleaned.to_csv("stroke_data_cleaned.csv", index=False)


In [14]:
print("age_catergory",stroke_dft['age_catergory'].unique().tolist())
print("age_catergory",stroke_dft['age_catergory'].nunique())
print("bmi_category",stroke_dft['bmi_category'].unique().tolist())
print("bmi_category",stroke_dft['bmi_category'].nunique())
print("gender",stroke_dft['gender'].unique().tolist())
print("gender",stroke_dft['gender'].nunique())
print("ever_married",stroke_dft['ever_married'].unique().tolist())
print("ever_married",stroke_dft['ever_married'].nunique())
print("work_type",stroke_dft['work_type'].unique().tolist())
print("work_type",stroke_dft['work_type'].nunique())
print("Residence_type",stroke_dft['Residence_type'].unique().tolist())
print("Residence_type",stroke_dft['Residence_type'].nunique())
print("smoking_status",stroke_dft['smoking_status'].unique().tolist())
print("smoking_status",stroke_dft['smoking_status'].nunique())


age_catergory ['Senior', 'Adult', 'Child', 'Young_Adult']
age_catergory 4
bmi_category ['Obese', 'Overweight', 'Normal', 'Underweight']
bmi_category 4
gender ['Male', 'Female', 'Other']
gender 3
ever_married ['Yes', 'No']
ever_married 2
work_type ['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked']
work_type 5
Residence_type ['Urban', 'Rural']
Residence_type 2
smoking_status ['formerly smoked', 'never smoked', 'smokes', 'Unknown']
smoking_status 4


Multiple Variable Linear Regression testing .

In [15]:
avoid_columns_regression  = [ 
 'ever_married',
 'Residence_type']
reg_analysis_dft  = stroke_dft_cleaned
reg_analysis_dft.drop(columns=avoid_columns_regression, inplace=True)
reg_analysis_dft.head(5)



,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,ever_married_bool,residence_type_bool,g_Female,g_Male,...,work_type_Self-employed,work_type_children,gl_catergory_Diabetic,gl_catergory_Low,gl_catergory_Normal,gl_catergory_Prediabetic,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,1,1,False,True,...,False,False,True,False,False,False,False,True,False,False
1,61.0,0,0,202.21,28.1,1,1,0,True,False,...,True,False,True,False,False,False,False,False,True,False
2,80.0,0,1,105.92,32.5,1,1,0,False,True,...,False,False,False,False,True,False,False,False,True,False
3,49.0,0,0,171.23,34.4,1,1,1,True,False,...,False,False,False,False,False,True,False,False,False,True
4,79.0,1,0,174.12,24.0,1,1,0,True,False,...,True,False,False,False,False,True,False,False,True,False


In [16]:
from imblearn.over_sampling import SMOTE

y_axis  = reg_analysis_dft['stroke']
# reg_analysis_dft.drop(columns=['stroke'], inplace=True)
print("count of person affected by stroke and count of people who dont have ",reg_analysis_dft.value_counts())
x_axis = reg_analysis_dft.copy()
x_axis.drop(columns=['stroke'], inplace=True)
# Apply SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(x_axis, y_axis)

print("Balanced dataset:")
print(pd.Series(y_balanced).value_counts())
print("X_balanced shape:", len(X_balanced))
print("y_balanced shape:",len(y_balanced))


count of person affected by stroke and count of people who dont have  age    hypertension  heart_disease  avg_glucose_level  bmi   stroke  ever_married_bool  residence_type_bool  g_Female  g_Male  g_Other  age_Adult  age_Child  age_Senior  age_Young_Adult  bmi_Normal  bmi_Obese  bmi_Overweight  bmi_Underweight  work_type_Govt_job  work_type_Never_worked  work_type_Private  work_type_Self-employed  work_type_children  gl_catergory_Diabetic  gl_catergory_Low  gl_catergory_Normal  gl_catergory_Prediabetic  smoking_status_Unknown  smoking_status_formerly smoked  smoking_status_never smoked  smoking_status_smokes
82.00  1             1              215.94             27.9  1       1                  1                    True      False   False    False      False      True        False            False       False      True            False            True                False                   False              False                    False               True                   False     

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, classification_report
# from sklearn.metrics import mean_absolute_error, r2_score-- i m not predicting  probabilities of values instead of boolean values meaning ocurrence of stroke or not
# stroke is possible or not possible ( so they are not continuous values)


scaler = StandardScaler()
num_cols = ['age', 'avg_glucose_level', 'bmi']
X_balanced[num_cols] = scaler.fit_transform(X_balanced[num_cols])

# Split data: train, validation, test (70/15/15)
X_train, X_temp, y_train, y_temp = train_test_split(X_balanced,y_balanced, test_size=0.3, stratify=y_balanced, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

mlr_model = LinearRegression()
mlr_model.fit(X_train, y_train)


val_preds = mlr_model.predict(X_val)
test_preds = mlr_model.predict(X_test)

# Convert to binary classification
val_preds_binary = np.round(val_preds)
test_preds_binary = np.round(test_preds)

val_rmse = root_mean_squared_error(y_val, val_preds)
test_rmse = root_mean_squared_error(y_test, test_preds)

print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")


print("Validation Report:")
print(classification_report(y_val, val_preds_binary))

print("Test Report:")
print(classification_report(y_test, test_preds_binary))

#  Overfitting Check
if abs(val_rmse - test_rmse) < 0.02:
    print(" No overfitting detected: validation and test performance are consistent.")
else:
    print(" Potential overfitting: consider regularization or more data.")

Validation RMSE: 0.2559
Test RMSE: 0.2547
Validation Report:
              precision    recall  f1-score   support

         0.0       0.90      0.99      0.94       729
         1.0       0.99      0.88      0.93       729
         2.0       0.00      0.00      0.00         0

    accuracy                           0.94      1458
   macro avg       0.63      0.62      0.62      1458
weighted avg       0.94      0.94      0.94      1458

Test Report:
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95       730
         1.0       0.99      0.90      0.95       729
         2.0       0.00      0.00      0.00         0

    accuracy                           0.95      1459
   macro avg       0.64      0.63      0.63      1459
weighted avg       0.95      0.95      0.95      1459

 No overfitting detected: validation and test performance are consistent.


d:\general_pyvenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\general_pyvenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\general_pyvenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\general_pyvenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: Undefine

logisitic  regression model

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

y_pred = log_model.predict(X_test)                # predicted labels (0 or 1)
y_pred_proba = log_model.predict_proba(X_test)    # predicted probabilities

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))




Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.98      0.96       730
           1       0.98      0.93      0.96       729

    accuracy                           0.96      1459
   macro avg       0.96      0.96      0.96      1459
weighted avg       0.96      0.96      0.96      1459

Confusion Matrix:
[[719  11]
 [ 51 678]]


In [19]:

new_person_raw = {
    'age': 67,
    'hypertension': 0,
    'heart_disease': 1,
    'avg_glucose_level': 228.69,
    'bmi': 36.6,
    'ever_married_bool': 1,
    'residence_type_bool': 1,
    'g_Female': 0,
    'g_Male': 1  ,
    'g_Other': 0 ,
    'age_Adult': 0,
    'age_Child': 0,
    'age_Senior': 1,
    'age_Young_Adult': 0,
    'bmi_Normal': 0,
    'bmi_Obese': 1,
    'bmi_Overweight': 0,
    'bmi_Underweight': 0,
    'work_type_Govt_job': 0,
    'work_type_Never_worked': 0,
    'work_type_Private': 1,
    'work_type_Self-employed': 0,
    'work_type_children': 0,
    'gl_catergory_Diabetic': 1,
    'gl_catergory_Low': 0,
    'gl_catergory_Normal': 0,
    'gl_catergory_Prediabetic': 0,
    'smoking_status_Unknown': 0,
    'smoking_status_formerly smoked': 1,
    'smoking_status_never smoked': 0,
    'smoking_status_smokes': 0
}


raw_input = pd.DataFrame([new_person_raw])
raw_input[['age', 'avg_glucose_level', 'bmi']] = scaler.transform(raw_input[['age', 'avg_glucose_level', 'bmi']])

#Predict stroke

print ("Linear Regression Model")
pred =mlr_model.predict(raw_input)[0]
risk = "High risk of stroke" if pred >= 0.5 else "Low risk of stroke"
print(f"Predicted stroke risk score: {pred:.4f} → {risk}")



print ("Logistic Regression Model")
risk_score = log_model.predict_proba(raw_input)[0][1]  
risk = "High risk of stroke" if risk_score >= 0.5 else "Low risk of stroke"
print(f"Predicted stroke risk score: {risk_score:.4f} → {risk}")





Linear Regression Model
Predicted stroke risk score: 0.2807 → Low risk of stroke
Logistic Regression Model
Predicted stroke risk score: 0.1807 → Low risk of stroke


Association Rule using Apriori rule 

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules



# Combine both lists
selected_columns  = ['g_Female',
 'g_Male',
 'g_Other',
 'age_Adult',
 'age_Child',
 'age_Senior',
 'age_Young_Adult',
 'hypertension',
 'heart_disease',
 'ever_married_bool',
 'residence_type_bool',
 'bmi_Normal',
 'bmi_Obese',
 'bmi_Overweight',
 'bmi_Underweight',
 'work_type_Govt_job',
 'work_type_Never_worked',
 'work_type_Private',
 'work_type_Self-employed',
 'work_type_children',
 'gl_catergory_Diabetic',
 'gl_catergory_Low',
 'gl_catergory_Normal',
 'gl_catergory_Prediabetic',
 'smoking_status_Unknown',
 'smoking_status_formerly smoked',
 'smoking_status_never smoked',
 'smoking_status_smokes',
  'stroke']

assocuiation_dft = reg_analysis_dft[selected_columns]

frequent_itemsets = apriori(assocuiation_dft, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

print ( rules[rules['consequents'].apply(lambda x: 'stroke' in str(x))].head(10))
stroke_rules = stroke_rules.sort_values(by='lift', ascending=False)
# print(stroke_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])




d:\general_pyvenv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, representativity, leverage, conviction, zhangs_metric, jaccard, certainty, kulczynski]
Index: []


KeyError: 'lift'